In [1]:
%%capture output
!pip install ipywidgets
!pip install jupyterlab_widgets
# !pip install nodejs
# !pip install npm
!pip install widgetsnbextension
!jupyter nbextension enable --py widgetsnbextension

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.IntSlider()

In [4]:
display(w)

IntSlider(value=0)

In [5]:
display(w)

IntSlider(value=0)

In [6]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

FloatText(value=0.0)

FloatSlider(value=0.0)

In [7]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='1',
    description='Number:',
    disabled=False,
)

Dropdown(description='Number:', options=('1', '2', '3'), value='1')

In [8]:
from os import listdir
from os.path import isfile, join

mypath = 'data'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

filesDropdown = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Users:',
    disabled=False,
)
itemsDropdown = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Items:',
    disabled=False,
)

display(filesDropdown)
display(itemsDropdown)

Dropdown(description='Users:', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

Dropdown(description='Items:', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

In [10]:
filesDropdown.value

'Prado_users_similarity.csv'